# 1 - Análise de consistência dos dados

Nessa fase, analisaremos se os dados estão fazendo sentido, se os campos estão completos e se há dados duplicados ou faltantes.

In [1]:
import pandas as pd
import numpy as np

## Importação dos dados

Primeiramente, importamos os dados:

In [2]:
# não tocaremos no conjunto de submissão

tweets_raw = pd.read_csv(
    r'../data/Train3Classes.csv',
)

In [3]:
tweets_raw.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1049721159292346368,Rio elege maior bancada policial de sua histór...,Tue Oct 09 18:00:01 +0000 2018,2,folha
1,1046251157025423360,fiquei tão triste quando eu vi o preço da câme...,Sun Sep 30 04:11:28 +0000 2018,0,:(
2,1041744620206653440,"Para Theresa May, seu plano para o Brexit é a ...",Mon Sep 17 17:44:06 +0000 2018,2,exame
3,1046937084727107589,caralho eu quero proteger a danielly em um pot...,Tue Oct 02 01:37:06 +0000 2018,0,:(
4,1047326854229778432,@SiCaetano_ viva o caos :),Wed Oct 03 03:25:55 +0000 2018,1,:)


In [4]:
tweets_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          95000 non-null  int64 
 1   tweet_text  95000 non-null  object
 2   tweet_date  95000 non-null  object
 3   sentiment   95000 non-null  int64 
 4   query_used  95000 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.6+ MB


## Transformações básicas

Vamos fazer algumas transformações básicas:

In [5]:
def mudar_tipos(df):
    df = df.copy()

    df['id'] = df['id'].astype('string')
    df['tweet_date'] = pd.to_datetime(df['tweet_date'])
    df['sentiment'] = df['sentiment'].astype('category')

    return df

def setar_index(df):
    # somente será aplicado após remção de duplicatas
    df = df.copy()
    df = df.set_index('id')
    return df

tweets = (tweets_raw
    .pipe(mudar_tipos)
)

In [6]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          95000 non-null  string             
 1   tweet_text  95000 non-null  object             
 2   tweet_date  95000 non-null  datetime64[ns, UTC]
 3   sentiment   95000 non-null  category           
 4   query_used  95000 non-null  object             
dtypes: category(1), datetime64[ns, UTC](1), object(2), string(1)
memory usage: 3.0+ MB


In [7]:
tweets.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1049721159292346368,Rio elege maior bancada policial de sua histór...,2018-10-09 18:00:01+00:00,2,folha
1,1046251157025423360,fiquei tão triste quando eu vi o preço da câme...,2018-09-30 04:11:28+00:00,0,:(
2,1041744620206653440,"Para Theresa May, seu plano para o Brexit é a ...",2018-09-17 17:44:06+00:00,2,exame
3,1046937084727107589,caralho eu quero proteger a danielly em um pot...,2018-10-02 01:37:06+00:00,0,:(
4,1047326854229778432,@SiCaetano_ viva o caos :),2018-10-03 03:25:55+00:00,1,:)


## Análise de consistência

Vamos analisar se a base está completa, sem dados faltantes.

In [8]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95000 entries, 0 to 94999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          95000 non-null  string             
 1   tweet_text  95000 non-null  object             
 2   tweet_date  95000 non-null  datetime64[ns, UTC]
 3   sentiment   95000 non-null  category           
 4   query_used  95000 non-null  object             
dtypes: category(1), datetime64[ns, UTC](1), object(2), string(1)
memory usage: 3.0+ MB


Não há dados faltantes.

Vejamos se há dados em que algum campo não está preenchido (ao invés de `np.nan`, pode estar como string nula, `''`)

In [9]:
tweets[
    (tweets.tweet_text == '') |
    (tweets.query_used == '')
]

,id,tweet_text,tweet_date,sentiment,query_used


In [10]:
tweets.sentiment.cat.categories

Int64Index([0, 1, 2], dtype='int64')

In [11]:
tweets.describe(datetime_is_numeric = True)

,id,tweet_text,tweet_date,sentiment,query_used
count,95000,95000,95000,95000.0,95000
unique,94987,94184,NaN,3.0,14
top,1040377082189172738,Bom dia :),NaN,0.0,:(
freq,2,30,NaN,31696.0,31696
mean,NaN,NaN,2018-09-28 18:18:50.530284032+00:00,NaN,NaN
min,NaN,NaN,2018-08-09 01:59:00+00:00,NaN,NaN
25%,NaN,NaN,2018-09-27 17:50:36.500000+00:00,NaN,NaN
50%,NaN,NaN,2018-10-01 18:20:00+00:00,NaN,NaN
75%,NaN,NaN,2018-10-08 01:16:52+00:00,NaN,NaN
max,NaN,NaN,2018-10-19 18:40:47+00:00,NaN,NaN



Vamos ver se há dados duplicados:

In [12]:
duplicados = tweets[tweets.duplicated(subset = 'id', keep = False)]

with pd.option_context('display.max_colwidth', 500):
    display(duplicados.tweet_text)

4777                                Veja o que é #FATO ou #FAKE na entrevista de Fernando Haddad para o Jornal da Globo. https://t.co/v0yFR5LRC1 https://t.co/fnbcEnrz9j
17419                                                  Veja o que é #FATO ou #FAKE na sabatina de Marina Silva no GLOBO. https://t.co/KOZduYQanr https://t.co/DEidfc1pz5
18026                                                    Veja o que é #FATO ou #FAKE na sabatina de Ciro Gomes no GLOBO. https://t.co/jmPEAZcTbX https://t.co/79w5eSp4p3
20226                                                 Veja o que é #FATO ou #FAKE na entrevista de Eymael ao G1 e à CBN. https://t.co/SkbV38GESy https://t.co/Zc9GQzYCjp
21547                                                                    Veja o que é #FATO ou #FAKE na entrevista de Anthony Garotinho ao RJTV. https://t.co/9EpMm0jVwv
28564                               Veja o que é #FATO ou #FAKE na entrevista de Fernando Haddad para o Jornal Nacional. https://t.co/PAUdomrbUu https://t.

In [13]:
duplicados.shape

(26, 5)

Há 26 *tweets* duplicados. Vamos manter somente uma cópia de cada duplicata:

In [14]:
tweets.drop_duplicates(subset = 'id')

,id,tweet_text,tweet_date,sentiment,query_used
0,1049721159292346368,Rio elege maior bancada policial de sua histór...,2018-10-09 18:00:01+00:00,2,folha
1,1046251157025423360,fiquei tão triste quando eu vi o preço da câme...,2018-09-30 04:11:28+00:00,0,:(
2,1041744620206653440,"Para Theresa May, seu plano para o Brexit é a ...",2018-09-17 17:44:06+00:00,2,exame
3,1046937084727107589,caralho eu quero proteger a danielly em um pot...,2018-10-02 01:37:06+00:00,0,:(
4,1047326854229778432,@SiCaetano_ viva o caos :),2018-10-03 03:25:55+00:00,1,:)
...,...,...,...,...,...
94995,1041831666883321856,Cuba e defensor de direitos humanos se unem co...,2018-09-17 23:30:00+00:00,2,jornaloglobo
94996,1032352892194369536,#Oportunidade ➡️ Venha fazer parte da nossa eq...,2018-08-22 19:44:44+00:00,2,#oportunidade
94997,1046785538542440448,@96syoo EU SEI 😭😭 é por isso que significa mui...,2018-10-01 15:34:55+00:00,0,:(
94998,1045457469110177792,@louistsexhes N te conheço mas posta :D,2018-09-27 23:37:38+00:00,1,:)


In [15]:
def remover_duplicatas(df):
    df = df.copy()

    df = df.drop_duplicates(subset = 'id')

    return df

tweets = (tweets_raw
    .pipe(mudar_tipos)
    .pipe(remover_duplicatas)
    .pipe(setar_index)
)

In [16]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94987 entries, 1049721159292346368 to 1046239135286136832
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   tweet_text  94987 non-null  object             
 1   tweet_date  94987 non-null  datetime64[ns, UTC]
 2   sentiment   94987 non-null  category           
 3   query_used  94987 non-null  object             
dtypes: category(1), datetime64[ns, UTC](1), object(2)
memory usage: 3.0+ MB
